In [ ]:
!pip install pycytominer
!pip install anndata

In [1]:
# needed for import pycytominer to work
import scipy
from scipy.stats import median_abs_deviation
scipy.stats.median_absolute_deviation = median_abs_deviation
import anndata as ad
import pandas as pd
import numpy as np
import pycytominer as pc

In [4]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
import os
gdrive_path='/content/gdrive/MyDrive/Masterpraktikum'
drive.mount('/content/gdrive', force_remount=True)
os.chdir(gdrive_path)

Mounted at /content/gdrive


# Run Cytominer: Normalization and Feature Selection

1. Create Profiles
2. Normalize Features (MAD)
3. Select Features
4. Spherize

Read in the data:

In [3]:
adata_target2 = ad.read_h5ad("/content/gdrive/MyDrive/Masterpraktikum/Blockteil/jump_target2_imputed.h5ad")
adata_target2

AnnData object with n_obs × n_vars = 64464 × 4706
    obs: 'Metadata_JCP2022', 'Metadata_InChIKey', 'Metadata_InChI', 'Metadata_Source', 'Metadata_Plate', 'Metadata_Well', 'Metadata_PlateType', 'Metadata_WellType', 'Metadata_Microscope_Name', 'Metadata_Widefield_vs_Confocal', 'Metadata_Excitation_Type', 'Metadata_Objective_NA', 'Metadata_N_Brightfield_Planes_Min', 'Metadata_N_Brightfield_Planes_Max', 'Metadata_Distance_Between_Z_Microns', 'Metadata_Sites_Per_Well', 'Metadata_Filter_Configuration', 'Metadata_Excitation_Low_DNA', 'Metadata_Excitation_Low_ER', 'Metadata_Excitation_Low_RNA', 'Metadata_Excitation_Low_AGP', 'Metadata_Excitation_Low_Mito', 'Metadata_Excitation_High_DNA', 'Metadata_Excitation_High_ER', 'Metadata_Excitation_High_RNA', 'Metadata_Excitation_High_AGP', 'Metadata_Excitation_High_Mito', 'Metadata_Emission_Low_DNA', 'Metadata_Emission_Low_ER', 'Metadata_Emission_Low_RNA', 'Metadata_Emission_Low_AGP', 'Metadata_Emission_Low_Mito', 'Metadata_Emission_High_DNA', 'Metada

## 1. Create Profiles

**Create pandas dataframe for normalization**

In [6]:
metadata = adata_target2.obs
metadata.head()

,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_PlateType,Metadata_WellType,Metadata_Microscope_Name,Metadata_Widefield_vs_Confocal,...,Metadata_Emission_High_Mito,Metadata_FPBase_Config,broad_sample,InChIKey,pert_iname,pubchem_cid,target,pert_type,control_type,smiles
0,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210726-160150,G21,TARGET2,treatment,CV8000,Confocal,...,690.5,http://broad.io/JUMPProductionConfigCV8000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210727-153003,G21,TARGET2,treatment,CV8000,Confocal,...,690.5,http://broad.io/JUMPProductionConfigCV8000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210803-153958,G21,TARGET2,treatment,CV8000,Confocal,...,690.5,http://broad.io/JUMPProductionConfigCV8000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210809-134534,G21,TARGET2,treatment,CV8000,Confocal,...,690.5,http://broad.io/JUMPProductionConfigCV8000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210810-173723,G21,TARGET2,treatment,CV8000,Confocal,...,690.5,http://broad.io/JUMPProductionConfigCV8000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
features = pd.DataFrame(adata_target2.X, columns = adata_target2.var.index)
features

,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,2972.4,6387.4,586.37,560.34,507.18,480.19,546.72,519.66,2.6556,0.82131,...,46.7820,40.8260,47.0480,48.2570,46.9350,48.1520,48.7950,50.1630,49.1250,50.1740
1,3134.5,6601.0,508.22,567.50,429.44,484.90,468.39,525.72,2.6599,0.82976,...,25.6070,23.1280,26.6730,27.1620,26.3630,27.1780,27.6340,28.0500,27.3910,28.1970
2,2995.2,6406.1,538.87,548.18,460.60,466.81,499.39,507.03,2.6466,0.83256,...,38.0640,33.3610,37.5740,38.2970,37.4180,38.3690,38.7800,39.4270,38.7050,39.5830
3,3017.5,6358.2,563.13,537.36,482.72,458.59,522.38,497.54,2.5986,0.82182,...,45.7140,40.3380,46.0870,47.0210,45.9320,47.1160,47.6800,48.5670,47.5610,48.6770
4,2995.9,6493.9,532.85,587.49,454.79,505.73,493.45,546.03,2.4955,0.81958,...,52.7130,48.5420,52.4000,53.4810,52.0620,53.2910,54.2230,55.4600,53.9870,55.3660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64459,3303.7,6153.5,514.84,519.03,438.32,440.31,475.96,479.07,2.3107,0.76807,...,2.3945,2.3296,2.2707,2.2821,2.2672,2.2987,2.3121,2.3585,2.3094,2.3774
64460,3376.2,6471.1,535.32,538.05,454.79,459.05,494.61,497.94,2.3174,0.78317,...,2.3402,2.1777,2.1627,2.1811,2.1544,2.1919,2.2161,2.2842,2.2124,2.2870
64461,3254.0,6156.9,529.68,528.15,452.22,450.04,490.52,488.56,2.3413,0.77440,...,2.1538,2.0642,2.0183,2.0271,2.0083,2.0304,2.0459,2.1126,2.0449,2.1013
64462,3188.8,6090.0,522.20,519.79,445.16,442.14,483.34,480.39,2.3344,0.77956,...,3.1756,2.9922,2.9419,2.9699,2.9372,2.9784,3.0106,3.1286,2.9978,3.1076


In [8]:
features.index = range(64464)
metadata.index = range(64464)

In [9]:
# merge both data frames:
features = pd.merge(metadata, features, left_index = True, right_index = True)
features

,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_PlateType,Metadata_WellType,Metadata_Microscope_Name,Metadata_Widefield_vs_Confocal,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210726-160150,G21,TARGET2,treatment,CV8000,Confocal,...,46.7820,40.8260,47.0480,48.2570,46.9350,48.1520,48.7950,50.1630,49.1250,50.1740
1,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210727-153003,G21,TARGET2,treatment,CV8000,Confocal,...,25.6070,23.1280,26.6730,27.1620,26.3630,27.1780,27.6340,28.0500,27.3910,28.1970
2,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210803-153958,G21,TARGET2,treatment,CV8000,Confocal,...,38.0640,33.3610,37.5740,38.2970,37.4180,38.3690,38.7800,39.4270,38.7050,39.5830
3,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210809-134534,G21,TARGET2,treatment,CV8000,Confocal,...,45.7140,40.3380,46.0870,47.0210,45.9320,47.1160,47.6800,48.5670,47.5610,48.6770
4,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210810-173723,G21,TARGET2,treatment,CV8000,Confocal,...,52.7130,48.5420,52.4000,53.4810,52.0620,53.2910,54.2230,55.4600,53.9870,55.3660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64459,JCP2022_116749,ZZZRUAITSXLWBH-UHFFFAOYSA-N,InChI=1S/C23H27N3O7/c1-25(2)12-5-6-13(27)15-10...,source_9,GR00004395,Y37,TARGET2,treatment,Opera Phenix,Widefield,...,2.3945,2.3296,2.2707,2.2821,2.2672,2.2987,2.3121,2.3585,2.3094,2.3774
64460,JCP2022_116749,ZZZRUAITSXLWBH-UHFFFAOYSA-N,InChI=1S/C23H27N3O7/c1-25(2)12-5-6-13(27)15-10...,source_9,GR00004409,I13,TARGET2,treatment,Opera Phenix,Widefield,...,2.3402,2.1777,2.1627,2.1811,2.1544,2.1919,2.2161,2.2842,2.2124,2.2870
64461,JCP2022_116749,ZZZRUAITSXLWBH-UHFFFAOYSA-N,InChI=1S/C23H27N3O7/c1-25(2)12-5-6-13(27)15-10...,source_9,GR00004409,I37,TARGET2,treatment,Opera Phenix,Widefield,...,2.1538,2.0642,2.0183,2.0271,2.0083,2.0304,2.0459,2.1126,2.0449,2.1013
64462,JCP2022_116749,ZZZRUAITSXLWBH-UHFFFAOYSA-N,InChI=1S/C23H27N3O7/c1-25(2)12-5-6-13(27)15-10...,source_9,GR00004409,Y13,TARGET2,treatment,Opera Phenix,Widefield,...,3.1756,2.9922,2.9419,2.9699,2.9372,2.9784,3.0106,3.1286,2.9978,3.1076


In [10]:
features.to_csv("/content/gdrive/MyDrive/Masterpraktikum/Blockteil/target2_profiles.csv", compression='gzip')

## 2. Normalize Features

Available normalization methods:
- "standardize"
  - uses sklearn.preprocessing StandardScaler
  - z = (x - mean) / sd
- "robustize"
  - uses sklearn.preprocessing RobustScaler
    - Scale features using statistics that are robust to outliers
    - scaled = (x - median) / IQR
    
      (IQR == Interquartile Range)
- "mad_robustize"
  - scaled = (x - median) / mad
- "spherize"
  - transforms into vectors whose covariance is the identity matrix
  - -> meaning that they are uncorrelated and each have variance 1

**Chosen method == "mad_robustize", plate-wise**

In [2]:
profiles = pd.read_csv("/content/gdrive/MyDrive/Masterpraktikum/Blockteil/target2_profiles.csv" , compression='gzip', index_col = 0)
profiles.head()

<ipython-input-2-edab4eea458a>:1: DtypeWarning: Columns (5,39,40,41,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  profiles = pd.read_csv("/content/gdrive/MyDrive/Masterpraktikum/Blockteil/target2_profiles.csv" , compression='gzip', index_col = 0)


,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_PlateType,Metadata_WellType,Metadata_Microscope_Name,Metadata_Widefield_vs_Confocal,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210726-160150,G21,TARGET2,treatment,CV8000,Confocal,...,46.782,40.826,47.048,48.257,46.935,48.152,48.795,50.163,49.125,50.174
1,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210727-153003,G21,TARGET2,treatment,CV8000,Confocal,...,25.607,23.128,26.673,27.162,26.363,27.178,27.634,28.050,27.391,28.197
2,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210803-153958,G21,TARGET2,treatment,CV8000,Confocal,...,38.064,33.361,37.574,38.297,37.418,38.369,38.780,39.427,38.705,39.583
3,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210809-134534,G21,TARGET2,treatment,CV8000,Confocal,...,45.714,40.338,46.087,47.021,45.932,47.116,47.680,48.567,47.561,48.677
4,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210810-173723,G21,TARGET2,treatment,CV8000,Confocal,...,52.713,48.542,52.400,53.481,52.062,53.291,54.223,55.460,53.987,55.366


In [3]:
metadata = ['Metadata_JCP2022', 'Metadata_InChIKey', 'Metadata_InChI', 'Metadata_Source', 'Metadata_Plate', 'Metadata_Well', 'Metadata_PlateType', 'Metadata_WellType', 'Metadata_Microscope_Name', 'Metadata_Widefield_vs_Confocal', 'Metadata_Excitation_Type', 'Metadata_Objective_NA', 'Metadata_N_Brightfield_Planes_Min', 'Metadata_N_Brightfield_Planes_Max', 'Metadata_Distance_Between_Z_Microns', 'Metadata_Sites_Per_Well', 'Metadata_Filter_Configuration', 'Metadata_Excitation_Low_DNA', 'Metadata_Excitation_Low_ER', 'Metadata_Excitation_Low_RNA', 'Metadata_Excitation_Low_AGP', 'Metadata_Excitation_Low_Mito', 'Metadata_Excitation_High_DNA', 'Metadata_Excitation_High_ER', 'Metadata_Excitation_High_RNA', 'Metadata_Excitation_High_AGP', 'Metadata_Excitation_High_Mito', 'Metadata_Emission_Low_DNA', 'Metadata_Emission_Low_ER', 'Metadata_Emission_Low_RNA', 'Metadata_Emission_Low_AGP', 'Metadata_Emission_Low_Mito', 'Metadata_Emission_High_DNA', 'Metadata_Emission_High_ER', 'Metadata_Emission_High_RNA', 'Metadata_Emission_High_AGP', 'Metadata_Emission_High_Mito', 'Metadata_FPBase_Config', 'broad_sample', 'InChIKey', 'pert_iname', 'pubchem_cid', 'target', 'pert_type', 'control_type', 'smiles']

In [ ]:
for plate in profiles["Metadata_Plate"].unique():
  profiles.loc[profiles["Metadata_Plate"] == plate] = pc.normalize(profiles[profiles["Metadata_Plate"] == plate],
                       meta_features = metadata,
                       method = "mad_robustize",
                       samples = "Metadata_WellType == 'DMSO'")

In [11]:
profiles.to_csv("/content/gdrive/MyDrive/Masterpraktikum/Blockteil/target2_profiles_normalized.csv", compression='gzip')

In [12]:
profiles.head()

,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_PlateType,Metadata_WellType,Metadata_Microscope_Name,Metadata_Widefield_vs_Confocal,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210726-160150,G21,TARGET2,treatment,CV8000,Confocal,...,-0.425339,-0.865407,-0.391177,-0.377283,-0.504986,-0.388730,-0.361701,-0.319453,-0.405932,-0.289039
1,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210727-153003,G21,TARGET2,treatment,CV8000,Confocal,...,-2.643884,-2.543497,-2.579051,-2.832310,-2.873830,-2.801176,-2.769876,-2.823216,-2.807881,-2.775110
2,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210803-153958,G21,TARGET2,treatment,CV8000,Confocal,...,41.439050,39.306349,46.651977,46.993913,47.276215,47.501270,46.508137,45.410042,47.283993,46.169697
3,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210809-134534,G21,TARGET2,treatment,CV8000,Confocal,...,-0.274125,-0.305272,-0.239266,-0.242540,-0.237972,-0.236065,-0.242079,-0.244885,-0.238958,-0.240313
4,JCP2022_000794,AECDBHGVIIRMOI-UHFFFAOYSA-N,InChI=1S/C27H29N5O/c28-26-25-24(21-8-4-9-23(14...,source_10,Dest210810-173723,G21,TARGET2,treatment,CV8000,Confocal,...,-0.364989,-0.415351,-0.290397,-0.261996,-0.295887,-0.278167,-0.262126,-0.261278,-0.268434,-0.268839


## 4. Select Features